## Basic classifier that uses Bidirectional Gated Recurrent Unit (bigru) to predict if the payload is an SQL injection

Let's import our stuff here


In [ ]:
!pip3 install scikit-learn>=1.0.0
!pip3 install ktrain matplotlib tensorflow
import matplotlib
import os
%reload_ext autoreload
%autoreload 2
%matplotlib inline
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 

Some more imports...We are using ktrain wrapper to simplify model operations and take advantage of some cool stuff like autofit

In [ ]:
import ktrain
from ktrain import text

Here, we will classify payload to "malicious sqli" and "benign" . Data set is presented as a CSV file (i.e., download the file ```SQLiV3a_.csv```).  We will load the data using the ```texts_from_csv``` method, which assumes the label_columns are already one-hot-encoded in the spreadsheet. Since *val_filepath* is None, 10% of the data will automatically be used as a validation set.


In [ ]:
DATA_PATH = 'SQLiV3a_8k.csv'
NUM_WORDS = 50000
MAXLEN = 100
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_csv(DATA_PATH,
                      'payload',
                      label_columns = ["class"],
                      val_filepath='SQLiV3a_4k.csv', # if None, 10% of data will be used for validation
                      max_features=NUM_WORDS, maxlen=MAXLEN,
                      ngram_range=1,
                      preprocess_mode='standard')

Load the learner instance that uses ```bigru``` model 

In [ ]:
model = text.text_classifier('bigru', (x_train, y_train), 
                             preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), val_data=(x_test, y_test))

Now let's find the optimal learning rate for this model

In [ ]:
learner.lr_find()
learner.lr_plot()

Train the model using the optimal learning rate ( adjust argument as needed accorfing to the graph)

In [ ]:
learner.autofit(0.01)

Now let's evaluate the the ```validation data set```

In [ ]:
learner.evaluate(class_names=['benign','malicious'])

In [ ]:
learner.predict()

Here is what our model looks like. It has a number of layers that comprise this ```GRU```

In [ ]:
learner.print_layers()


In [ ]:
learner.freeze()

In [ ]:
learner.save_model('models')

It's time for some fun! First, get a predictor instance that uses our pre-trained model

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

Let's see if it can catch an SQLi

In [ ]:
predictor.predict("1'||  (  select 'bgdh' from dual where 1967  =  1967")

In [ ]:
predictor.predict("/?ident=500")